In [25]:
import numpy as np
import functools

## Еталони

In [26]:
zero = np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 0, 1],
                 [1, 0, 1],
                 [1, 1, 1]])

one  = np.array([[0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0]])

two  = np.array([[1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1]])

three =np.array([[1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1]])

four = np.array([[1, 0, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [0, 0, 1]])

five = np.array([[1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1]])

six =  np.array([[1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1]])

seven =np.array([[1, 1, 1],
                 [0, 0, 1],
                 [0, 1, 0],
                 [1, 0, 0],
                 [1, 0, 0]])

eight =np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1]])


nine = np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [0, 0, 1]])

In [27]:
Y = [zero, one, two, three, four, five, six, seven, eight, nine] # еталонні зображення

## Генеруємо гістограми

In [29]:
histogram = np.random.randint(0,100,10)   # change value of seed if you want to get new histogram
histogram = histogram/np.sum(histogram)  # normilizing of histogram

## Генеруємо вхідне зображення
### повертаємо зобр. і суму цифр

In [87]:
def task(Y, t, p):
    r = np.random.choice(10,t,p=histogram)
    x_etalone = Y[r[0]]
    for i in range(1,t):
        x_etalone = np.concatenate((x_etalone,Y[r[i]]),axis=1)
    n = Y[0].shape[0]
    m = Y[0].shape[1]
    ksi = np.random.binomial(size=n*m*t, n=1, p=p).reshape((n,m*t))
    f = (x_etalone^ksi, sum(r)) 
    return f

## Рахуємо ймовірності

In [31]:
def recognition(task,noise_level, numbers):        # функція з першої лаби для апостеріорних ймовірностей
    result = []
    for i in numbers:
        d = []
        noise = i^task          
        
        noise = noise.reshape(1, noise.shape[0]*noise.shape[1])
        
        for j in range(0, noise.shape[0]*noise.shape[1]):           
            if noise[0][j] == 0:
                d.append(1-noise_level)
            else:
                d.append(noise_level)

        result.append(np.prod(d))              
    return result

## Рекурсивно рахуємо ймов-ті для під-зображень

In [32]:
@functools.lru_cache(None)
def f(t):
    if t==1:
        return recognition(im[t-1],noise,Y)*histogram
    else:
        res = []
    for d in range(0,9*t+1):
        k2 = []
        for k in range (0,10):
        
            if k>=d-9*(t-1) and k<=d:
                x2 = im[t-1]
                p22 = recognition(x2,noise,Y)[k]*histogram[k]
                k2.append( f(t-1)[d-k]*p22)
        res.append(np.sum(k2))
    return res

## Запускаємо цикл для перевірки

In [96]:
noise = 0
t = 15
a = []
b = []
for i in range(0,10):
    task1 = task(Y,t,noise)
    x = task1[0] # вхідне зображення
    im = np.hsplit(x, t)    # вхідне число(зображення) розбите на цифри 
    a.append(np.argmax(f(t))) # вгадана сума
    b.append(task1[1]) # реальна сума
    print("guessed_sum",a[i],'real_sum',b[i])
    f.cache_clear()  # очистка кешу

guessed_sum 62 real_sum 62
guessed_sum 53 real_sum 53
guessed_sum 58 real_sum 58
guessed_sum 60 real_sum 60
guessed_sum 76 real_sum 76
guessed_sum 41 real_sum 41
guessed_sum 72 real_sum 72
guessed_sum 53 real_sum 53
guessed_sum 60 real_sum 60
guessed_sum 64 real_sum 64


## Точність (кількість вгаданих з усіх)

In [97]:
c = 0
for i in range(len(a)):
    if a[i] == b[i]:
        c+=1
print(c,'/',len(a))

10 / 10
